In [ ]:
import os
import numpy as np
from scipy.fft import rfft, irfft, rfftfreq
from pywt import cwt, wavedec, waverec, swt, WaveletPacket
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams.update({"font.size": 12, "font.weight": 'bold', "lines.linewidth": 1.5})
# os.chdir(r"C:\Users\gs582\Dropbox\Publications\Journal of Magnetic resonance\swt_fig")
#%matplotlib widget

In [ ]:
def data_wcfs(data, wave, lev):
    """Returns the approximation and detail of coefficients of the data at all the decomposition level using "wavedec".

    Args:
        data (Numpy Array): 1D vector of the reference data.
        wave (String): Wavelet to be used for wavelet decomposition.
        lev (Intiger): Deomposition level till which signal will be decomposed.

    Returns:
        cf (List): Returns a list of approximation and detail coefficients.
    """

    app_ref, app_rec, det_ref, det_rec, cf, cf_rec = [], [], [], [], [], []
    for i in range(1, lev+1):
        c = wavedec(data=data, wavelet = wave, level=i)
        cc = wavedec(data=data, wavelet = wave, level=i)
               
        app_ref.append(c[0])
        det_ref.append(cc[1])
        # Reconstruct the data using approximation
        for j in range(i):
            c[j+1][::] = 0
        app_rec.append(waverec(c, wavelet=wave))

        cc[0][::] = 0
        if i > 1:
            for j in range(1, i):
                cc[1+j][::] = 0
        det_rec.append(waverec(cc, wavelet=wave))
        
    cf.append(app_ref), cf.append(det_ref)
    cf_rec.append(app_rec), cf_rec.append(det_rec)

    return cf, cf_rec

In [ ]:
# Load the data
data_esr = np.loadtxt(r"C:\Users\gs582\Dropbox\Publications\PdF\Journal of Magnetic resonance\Data\esr_sim.txt")
data_esr30 = -1*np.loadtxt(r"C:\Users\gs582\Dropbox\Publications\PdF\Journal of Magnetic resonance\Data\esr_sim30.txt")

# Calculate number of sampling pint, sampling frequency
N, deltaT = len(data_esr[:, 0]), (data_esr[1, 0]-data_esr[0, 0])

# Take Fourier transform of the data
fft_esr, fft_esr30 = rfft(-1*data_esr[:, 1]), rfft(data_esr30)
fq = rfftfreq(N, deltaT)

# Take Short Time Fourier Transform of the data
# stft_fq, stft_t, stft_esr = stft(x=-1*data_esr[:, 1], fs=deltaT, window='hann', nperseg=256, noverlap=0)
# _, _, stft_esr30 = stft(x=data_esr30, fs=deltaT, window='hann', nperseg=256, noverlap=0)
seg = [0, 256, 512, 768]
stft_esr, stft_esr30 = np.empty((len(seg), len(fft_esr)), dtype=np.complex64), np.empty((len(seg), len(fft_esr)), dtype=np.complex64)

for i in range(len(seg)):
    tempd1, tempd2 = np.zeros((N)), np.zeros((N))
    tempd1[:256], tempd2[:256] = -1*data_esr[seg[i]:seg[i]+256, 1], data_esr30[seg[i]:seg[i]+256]
    stft_esr[i, :], stft_esr30[i, :] = rfft(tempd1), rfft(tempd2)

In [ ]:
# Frequency filtering
# Copy the data
fftbp_esr, fftbp_esr30 = np.copy(fft_esr), np.copy(fft_esr30)
fftgp_esr, fftgp_esr30 = np.copy(fft_esr), np.copy(fft_esr30)

# Perform low and high frequency filtering
#fftbp_esr[(freq_esr < np.max(freq_esr)/2)], fftbp_esr30[(freq_esr < np.max(freq_esr)/2)] = 0, 0
fftbp_esr30[(fq > (np.max(fq))/2)] = 0

# generate a Gaussian bandpass function
gaussp = np.exp(-((fq - min(fq))/(2*0.5))**2) + np.exp(-((fq - max(fq))/(2*0.5))**2)  # Possitive frequency
#gaussn = np.exp(-((fq + min(fq))/(2*0.1))**2) + np.exp(-((fq + max(fq))/(2*0.1))**2)  # Negative frequency
gaussf = gaussp #+ gaussn   # Only have possitive frequencies hence need to filter them only
fftgp_esr30 = fftgp_esr30*gaussf

# Reconstruct the data using inverse Fourier transform
databp_esr, databp_esr30 = irfft(fftbp_esr), irfft(fftbp_esr30)
datagp_esr, datagp_esr30 = irfft(fftgp_esr), irfft(fftgp_esr30)

# Take FT of the denoised data
ftbp_esr30, ftgp_esr30 = rfft(databp_esr30), rfft(datagp_esr30)

In [ ]:
# Plot data, FT and STFT
fig = plt.figure(figsize=(17, 6), dpi=100, layout='constrained')
gs = fig.add_gridspec(2, 4, hspace=0, wspace=0)
ax = gs.subplots(sharex=False, sharey=False)

ax[0, 0].plot(data_esr[:, 0], data_esr30, '-r', label='Noisy'), ax[0, 0].plot(data_esr[:, 0], -1*data_esr[:, 1], '-b', label='Noise-free')
ax[0, 0].legend(frameon=False, fontsize=10), ax[0, 0].set_xlim([(data_esr[0, 0]), (data_esr[1023, 0])])
ax[0, 0].set_xlabel('Magnetic Field ($Gauss$)', fontsize=14, fontweight='bold'), ax[0, 0].set_ylabel('Magnitude', fontsize=14, fontweight='bold')
ax[0, 0].set_title('Signal Domain')

ax[0, 1].plot(fq, abs(fft_esr30), '-r', label='Noise-free'), ax[0, 1].plot(fq, abs(fft_esr), '-b', label='Noisy')
ax[0, 1].legend(frameon=False, fontsize=10), ax[0, 1].set_xlim([fq[0], fq[len(fq)-1]])
ax[0, 1].set_xlabel('Frequency ($Gauss^{-1}$)', fontsize=14, fontweight='bold'), ax[0, 1].set_ylabel('Magnitude', fontsize=14, fontweight='bold')
ax[0, 1].set_title('Fourier Domain')

ax[1, 0].plot(data_esr[:, 0], databp_esr30, '-r', label='Noisy'), ax[1, 0].plot(data_esr[:, 0], databp_esr, '-b', label='Noise-free')
ax[1, 0].legend(frameon=False, fontsize=10), ax[1, 0].set_xlim([(data_esr[0, 0]), (data_esr[1023, 0])])
ax[1, 0].set_xlabel('Magnetic Field ($Gauss$)', fontsize=14, fontweight='bold'), ax[1, 0].set_ylabel('Magnitude', fontsize=14, fontweight='bold')
ax[1, 0].set_title('Highpass Filtering')

ax[1, 1].plot(data_esr[:, 0], datagp_esr30, '-r', label='Noisy'), ax[1, 1].plot(data_esr[:, 0], datagp_esr, '-b', label='Noise-free')
ax[1, 1].legend(frameon=False, fontsize=10), ax[1, 1].set_xlim([(data_esr[0, 0]), (data_esr[1023, 0])])
ax[1, 1].set_xlabel('Magnetic Field ($Gauss$)', fontsize=14, fontweight='bold'), ax[1, 1].set_ylabel('Magnitude', fontsize=14, fontweight='bold')
ax[1, 1].set_title('Gaussian Filtering')

ax[0, 2].plot(fq, abs(stft_esr30[0, :]), '-r', label='Noisy'), ax[0, 2].plot(fq, abs(stft_esr[0, :]), '-b', label='Noise-free')
ax[0, 2].legend(frameon=False, fontsize=10), ax[0, 2].set_xlim([fq[0], fq[len(fq)-1]])
ax[0, 2].set_xlabel('Frequency ($Gauss^{-1}$)', fontsize=14, fontweight='bold'), ax[0, 2].set_ylabel('Magnitude', fontsize=14, fontweight='bold')
ax[0, 2].set_title('STFT')

ax[0, 3].plot(fq, abs(stft_esr30[1, :]), '-r', label='Noisy'), ax[0, 3].plot(fq, abs(stft_esr[1, :]), '-b', label='Noise-free')
ax[0, 3].legend(frameon=False, fontsize=10), ax[0, 3].set_xlim([fq[0], fq[len(fq)-1]])
ax[0, 3].set_xlabel('Frequency ($Gauss^{-1}$)', fontsize=14, fontweight='bold'), ax[0, 3].set_ylabel('Magnitude', fontsize=14, fontweight='bold')
ax[0, 3].set_title('STFT')

ax[1, 2].plot(fq, abs(stft_esr30[2, :]), '-r', label='Noisy'), ax[1, 2].plot(fq, abs(stft_esr[2, :]), '-b', label='Noise-free')
ax[1, 2].legend(frameon=False, fontsize=10), ax[1, 2].set_xlim([fq[0], fq[len(fq)-1]])
ax[1, 2].set_xlabel('Frequency ($Gauss^{-1}$)', fontsize=14, fontweight='bold'), ax[1, 2].set_ylabel('Magnitude', fontsize=14, fontweight='bold')
ax[1, 2].set_title('STFT')

ax[1, 3].plot(fq, abs(stft_esr30[3, :]), '-r', label='Noisy'), ax[1, 3].plot(fq, abs(stft_esr[3, :]), '-b', label='Noise-free')
ax[1, 3].legend(frameon=False, fontsize=10), ax[1, 3].set_xlim([fq[0], fq[len(fq)-1]])
ax[1, 3].set_xlabel('Frequency ($Gauss^{-1}$)', fontsize=14, fontweight='bold'), ax[1, 3].set_ylabel('Magnitude', fontsize=14, fontweight='bold')
ax[1, 3].set_title('STFT')

plt.show()

In [ ]:
# fig = plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
# gs = fig.add_gridspec(1, 1, hspace=0, wspace=0)
# ax = gs.subplots(sharex=False, sharey=False)
# ax.plot(stft_fq)
# plt.show()

In [ ]:
plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
plt.plot(data_esr[:, 0],data_esr30, '-r', label='Noisy')
plt.plot(data_esr[:, 0], -1*data_esr[:, 1], '-b', label='Noise-free')
plt.legend(frameon=False, fontsize=10, loc='upper right')
plt.xlim([(data_esr[0, 0]), (data_esr[1023, 0])])
plt.xlabel('Index', fontsize=14, fontweight='bold'), plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
# plt.vlines(x=data_esr[255, 0], ymin=min(data_esr[:, 1])-0.01, ymax=max(data_esr[:, 1])+0.01, color='k', ls='--', lw=2)
# plt.vlines(x=data_esr[511, 0], ymin=min(data_esr[:, 1])-0.01, ymax=max(data_esr[:, 1])+0.01, color='k', ls='--', lw=2)
# plt.vlines(x=data_esr[767, 0], ymin=min(data_esr[:, 1])-0.01, ymax=max(data_esr[:, 1])+0.01, color='k', ls='--', lw=2)

# plt.savefig('esr.png', dpi=400, bbox_inches='tight', pad_inches=0.02)
plt.show()

In [ ]:
# Fourier transform plot
# Arrow properties
arprop, alp = dict(facecolor='black', shrink=0.005, width=2.5, alpha=0.7), 0.7

plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
plt.plot(fq, abs(fft_esr30), '-r', label='Noisy')
# plt.plot(fq, abs(fft_esr), '-b', label='Noise-free')
# plt.legend(frameon=False, fontsize=10, loc='upper right')
plt.xlim([fq[0], fq[len(fq)-1]])
plt.ylim([-0.1, 7.5])
plt.xlabel('Frequency ($Gauss^{-1}$)', fontsize=14, fontweight='bold')
plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
# plt.annotate('Signal \n Distortion', xy=(0.85, 3.5), xytext=(0.85, 6.6), xycoords='data', arrowprops=arprop, alpha=alp, ha='center', fontsize=10)
# plt.annotate('Unable to Remove \n all Noise', xy=(3.5, 2), xytext=(3.5, 5.0), xycoords='data', arrowprops=arprop, alpha=alp, ha='center', fontsize=10)
# plt.savefig('ftesrgp.png', dpi=400, transparent=True, bbox_inches='tight', pad_inches=0.02)
plt.show()

In [ ]:
# Short Time Fourier Transform plot
# Arrow properties
arprop, alp = dict(facecolor='black', shrink=0.005, width=2.5, alpha=0.7), 0.7

plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
plt.plot(fq, abs(stft_esr30[2, :]), '-r', label='Noisy')
plt.plot(fq, abs(stft_esr[2, :]), '-b', label='Noise-free')
plt.legend(frameon=False, fontsize=10, loc='upper right')
plt.xlim([fq[0], fq[len(fq)-1]])
plt.ylim([-0.01, 4.0])
plt.xlabel('Frequency ($Gauss^{-1}$)', fontsize=14, fontweight='bold')
plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
plt.annotate('Signal+Noise', xy=(0.7, 2.75), xytext=(0.7, 3.75), xycoords='data', arrowprops=arprop, alpha=alp, ha='center', fontsize=10)
plt.annotate('Noise Only', xy=(3.0, 1.5), xytext=(3.0, 2.5), xycoords='data', arrowprops=arprop, alpha=alp, ha='center', fontsize=10)
#plt.savefig('stftesr3.png', dpi=400, bbox_inches='tight', pad_inches=0.02)
plt.show()

In [ ]:
# Continuous Wavelet transform of the data
scale = np.arange(1, 45, 1)
cwt_esr, cfq_esr = cwt(data=-1*data_esr[:, 1], scales=scale, wavelet='gaus3')
cwt_esr30, cfq_esr30 = cwt(data=data_esr30, scales=scale, wavelet='gaus3')

In [ ]:
plt.figure(figsize=(4, 3), dpi=100, layout='constrained')
plt.contourf(data_esr[:, 0], cfq_esr, abs(cwt_esr)**0.5, cmap='hot')
#plt.contourf(data_esr[:, 0], cfq_esr30, abs(cwt_esr30)**0.5, cmap='hot')
plt.xlabel('Magnetic Field ($Gauss$)', fontsize=14, fontweight='bold')
#plt.ylabel('Scale', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.show()

In [ ]:
# Statonary wavelet transform
dl=4
cf_db6, cf30_db6 = swt(data=-1*data_esr[:, 1], wavelet='db6', level=dl), swt(data=data_esr30, wavelet='db6', level=dl)
cf_coif3, cf30_coif3 = swt(data=-1*data_esr[:, 1], wavelet='coif3', level=dl), swt(data=data_esr30, wavelet='coif3', level=dl)
cf_sym10, cf30_sym10 = swt(data=-1*data_esr[:, 1], wavelet='sym10', level=dl), swt(data=data_esr30, wavelet='sym10', level=dl)
cf_dmey, cf30_dmey = swt(data=-1*data_esr[:, 1], wavelet='dmey', level=dl), swt(data=data_esr30, wavelet='dmey', level=dl)

In [ ]:
dt, dt30 = cf_coif3, cf30_coif3
fig = plt.figure(figsize=(15, 4), dpi=100, layout='constrained')
gs = fig.add_gridspec(2, 4, hspace=0, wspace=0)
ax = gs.subplots(sharex=False, sharey=False)
for i in range(1, dl+1):
    ax[0, i-1].plot(dt30[-i][0], '-r', label='Noisy'), ax[0, i-1].set_xlim(0, 1023), ax[0, i-1].legend(frameon=False, fontsize=10, loc='upper right')
    ax[0, i-1].plot(dt[-i][0], '-b', label='Noise-free'), ax[0, i-1].set_xlim(0, 1023), ax[0, i-1].legend(frameon=False, fontsize=10, loc='upper right')

    ax[1, i-1].plot(dt30[-i][1], '-r', label='Noisy'), ax[1, i-1].set_xlim(0, 1023), ax[1, i-1].legend(frameon=False, fontsize=10, loc='upper right')
    ax[1, i-1].plot(dt[-i][1], '-b', label='Noise-free'), ax[1, i-1].set_xlim(0, 1023), ax[1, i-1].legend(frameon=False, fontsize=10, loc='upper right')

plt.show()

In [ ]:
# Save wavelet coefficients
dt, dt30 = cf_db6, cf30_db6
for i in range(1, dl+1):
    plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
    plt.plot(dt30[-i][0], '-r', label='Noisy')
    plt.plot(dt[-i][0], '-b', label='Noise-free')
    plt.legend(frameon=False, loc='upper right')
    plt.xlim([0, 1023])
    plt.xlabel('Index', fontsize=14, fontweight='bold')
    plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
    plt.ylim([min(dt[-i][0]), max(dt[-i][0])])
    #plt.xticks([])
    #plt.yticks([])
    # plt.savefig('esrappdb6_{i}.png'.format(i=i), dpi=400, bbox_inches='tight', pad_inches=0.02)

    plt.show()



In [ ]:
# Plot and save FT of wavelet coefficients
dt, dt30 = cf_coif3, cf30_coif3
for i in range(1, dl+1):
    plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
    plt.plot(fq, abs(rfft(dt30[-i][0])), '-r', label='Noisy', alpha=0.9)
    plt.plot(fq, abs(rfft(dt[-i][0])), '-b', label='Noise-free', alpha=0.9)
    plt.legend(frameon=False, fontsize=10)
    plt.xlim([fq[0], fq[len(fq)-1]])
    plt.xlabel('Frequency($Gauss^{-1}$)', fontsize=14, fontweight='bold')
    plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
    plt.ylim([min(abs(rfft(dt30[-i][0])))-1, max(abs(rfft(dt30[-i][0])))])
    #plt.annotate('Signal \n Distortion', xy=(0.85, 3.5), xytext=(0.85, 6.6), xycoords='data', arrowprops=arprop, alpha=alp, ha='center', fontsize=10)
    # plt.savefig('ftesrapp_{i}.png'.format(i=i), dpi=400, bbox_inches='tight', pad_inches=0.02)

    plt.show()

In [ ]:
wp_db6, wp_coif3 = WaveletPacket(data=-1*data_esr[:, 1], wavelet='db6', maxlevel=3), WaveletPacket(data=-1*data_esr[:, 1], wavelet='coif3', maxlevel=3)
wp30_db6, wp30_coif3 = WaveletPacket(data=data_esr30, wavelet='db6', maxlevel=3), WaveletPacket(data=data_esr30, wavelet='coif3', maxlevel=3)

In [ ]:
print([node.path for node in wp_db6.get_level(3, 'natural')])

In [ ]:
dt = wp30_coif3

fig = plt.figure(figsize=(17, 4), dpi=100, constrained_layout=True)
gs = fig.add_gridspec(3, 8, hspace=0, wspace=0)
ax1, ax2 = fig.add_subplot(gs[0, 0:4]), fig.add_subplot(gs[0, 4:])
ax3, ax4, ax5, ax6 = fig.add_subplot(gs[1, :2]), fig.add_subplot(gs[1, 2:4]),fig.add_subplot(gs[1, 4:6]), fig.add_subplot(gs[1, 6:])
for i in range(7, 15):
    exec(f'ax{i} = fig.add_subplot(gs[2, i-7])')
# ax7, ax8, ax9, ax10 = fig.add_subplot(gs[2, 0]), fig.add_subplot(gs[2, 1]),fig.add_subplot(gs[2, 2]), fig.add_subplot(gs[2, 3])
# ax11, ax12, ax13, ax14 = fig.add_subplot(gs[2, 4]), fig.add_subplot(gs[2, 5]),fig.add_subplot(gs[2, 6]), fig.add_subplot(gs[2, 7])

ax1.plot(dt['a'].data, '-b'), ax2.plot(dt['d'].data, '-r')
ax3.plot(dt['aa'].data, '-b'), ax4.plot(dt['ad'].data, '-r'), ax5.plot(dt['da'].data, '-b'), ax6.plot(dt['dd'].data, '-r')
ax7.plot(dt['aaa'].data, '-b'), ax8.plot(dt['aad'].data, '-r'), ax9.plot(dt['ada'].data, '-b'), ax10.plot(dt['add'].data, '-r')
ax11.plot(dt['daa'].data, '-b'), ax12.plot(dt['dad'].data, '-r'), ax13.plot(dt['dda'].data, '-b'), ax14.plot(dt['ddd'].data, '-r')

for i in range(1, 15):
    exec(f'ax{i}.set_xticks([])')
    exec(f'ax{i}.set_yticks([])')

plt.show()

In [ ]:
# Save wavelet packet transform plots
dt1, dt2 = wp30_coif3['ddd'].data, wp_coif3['ddd'].data
dt = wp30_coif3['aaa'].data
plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
plt.plot(dt1, '-r', label='Noisy')
plt.plot(dt2, '-b', label='Noise-free')
plt.legend(frameon=False, loc='upper right')
plt.xlim([0, len(dt)])
plt.xlabel('Index', fontsize=14, fontweight='bold')
plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
plt.ylim([min(dt), max(dt)])
#plt.title('A', fontsize=14, fontweight='bold')
# plt.xticks([])
# plt.yticks([])
# plt.savefig('wpesr30_ddd.png'.format(i=i), dpi=400, bbox_inches='tight', pad_inches=0.02)
# plt.savefig('wpesrnoise30_ddd.png'.format(i=i), dpi=400, bbox_inches='tight', pad_inches=0.02)

plt.show()

In [ ]:
# Discrete wavelet transform data
dl=1
cf_coif3, rcf_coif3 = data_wcfs(data=-1*data_esr[:, 1], wave='coif3', lev=dl)
cf30_coif3, rcf30_coif3 = data_wcfs(data=data_esr30, wave='coif3', lev=dl)

In [ ]:
# Save dwt coefficients plot
dt = cf30_coif3
for i in range(0, dl):
    plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
    plt.plot(dt[1][i], '-b')
    plt.xlim([0, len(dt[0][i])-1])
    plt.xlabel('Index', fontsize=14, fontweight='bold')
    plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
    # plt.ylim([min(dt[0][i]), max(dt[0][i])])
    #plt.savefig('dwtdb6detesr30_{i}.png'.format(i=i), dpi=400, bbox_inches='tight', pad_inches=0.02)

    plt.show()

In [ ]:
# Plot and save FT of wavelet coefficients
dt, dt30 = rcf_coif3, rcf30_coif3
for i in range(0, dl):
    #fq1 = rfftfreq(len(dt30[0][i]), deltaT)
    plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
    plt.plot(fq, abs(rfft(dt30[0][i])), '-b', label='Approximation')
    plt.plot(fq, abs(rfft(dt30[1][i])), '-r', label='Detail')
    plt.legend(frameon=False, fontsize=10)
    plt.xlim([fq[0], fq[len(fq)-1]])
    plt.xlabel('Frequency($Gauss^{-1}$)', fontsize=14, fontweight='bold')
    plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
    #plt.ylim([min(abs(rfft(dt[-i][0]))), max(abs(rfft(dt[-i][0])))])
    #plt.annotate('Signal \n Distortion', xy=(0.85, 3.5), xytext=(0.85, 6.6), xycoords='data', arrowprops=arprop, alpha=alp, ha='center', fontsize=10)
    #plt.savefig('dwtcoif3ftesr_{i}.png'.format(i=i), dpi=400, bbox_inches='tight', pad_inches=0.02)

    plt.show()

In [ ]:
# Inverse wavelet transform
# del wp1, wp2, rwp1, rwp2
wpdl = 3
wp1 = WaveletPacket(data=-1*data_esr[:, 1], wavelet='coif3', maxlevel=wpdl)
wp2 = WaveletPacket(data=-1*data_esr[:, 1], wavelet='coif3', maxlevel=wpdl)
wpn1 = WaveletPacket(data=data_esr30, wavelet='coif3', maxlevel=wpdl)
wpn2 = WaveletPacket(data=data_esr30, wavelet='coif3', maxlevel=wpdl)
#del wp1['d']

In [ ]:
# wp1['d'].data[::], wp1['a'].data[::] = 0, 0
# wp1['aa'].data[::], wp1['ad'].data[::], wp1['da'].data[::], wp1['dd'].data[::] = 0, 0, 0, 0
# wp1['aaa'].data[::], wp1['aad'].data[::], wp1['ada'].data[::], wp1['add'].data[::] = 0, 0, 0, 0
# wp1['daa'].data[::], wp1['dad'].data[::], wp1['ddd'].data[::] = 0, 0, 0
# del wp1['aa']

#del wp2['a']
# wp2['a'].data[::], wp2['d'].data[::] = 0, 0
# wp2['aa'].data[::], wp2['ad'].data[::], wp2['da'].data[::], wp2['dd'].data[::] = 0, 0, 0, 0
# wp2['aaa'].data[::], wp2['aad'].data[::], wp2['ada'].data[::], wp2['add'].data[::] = 0, 0, 0, 0
# wp2['daa'].data[::], wp2['dad'].data[::], wp2['dda'].data[::] = 0, 0, 0


In [ ]:
wpn1['d'].data[::], wpn1['a'].data[::] = 0, 0
wpn1['aa'].data[::], wpn1['ad'].data[::], wpn1['da'].data[::], wpn1['da'].data[::] = 0, 0, 0, 0
wpn1['aaa'].data[::], wpn1['aad'].data[::], wpn1['ada'].data[::], wpn1['add'].data[::] = 0, 0, 0, 0
wpn1['daa'].data[::], wpn1['dad'].data[::], wpn1['dda'].data[::] = 0, 0, 0
# del wp1['aa']

#del wp2['a']
wpn2['a'].data[::], wpn2['d'].data[::] = 0, 0
# wpn2['aa'].data[::], wpn2['ad'].data[::], wpn2['ad'].data[::] = 0, 0, 0
# wpn2['aaa'].data[::], wpn2['aad'].data[::], wpn2['ada'].data[::], wpn2['add'].data[::] = 0, 0, 0, 0
# wpn2['daa'].data[::], wpn2['dad'].data[::], wpn2['dda'].data[::] = 0, 0, 0


In [ ]:
# rwp1 = wp1.reconstruct(update=True)
# rwp2 = wp2.reconstruct(update=True)

rwpn1 = wpn1.reconstruct(update=True)
rwpn2 = wpn2.reconstruct(update=True)

In [ ]:
fig = plt.figure(figsize=(17, 4), dpi=100, constrained_layout=True)
gs = fig.add_gridspec(1, 2, hspace=0, wspace=0)
ax = gs.subplots(sharex=False, sharey=False)
ax[0].plot(wp1['dda'].data), ax[0].plot(wpn1['dda'].data)
ax[1].plot(wp2['ddd'].data), ax[1].plot(wpn1['ddd'].data)

plt.show()

In [ ]:
# Save FT of the reconstructed wavelet packet data
plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
plt.plot(fq, abs(rfft(rwpn1)), '-b', label='DDA')
plt.plot(fq, abs(rfft(rwpn2)), '-r', label='DDD')
plt.legend(frameon=False, fontsize=10)
plt.xlim([fq[0], fq[len(fq)-1]])
plt.ylim([-0.325, max(abs(rfft(rwpn1)))])
plt.xlabel('Frequency($Gauss^{-1}$)', fontsize=14, fontweight='bold')
plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
# plt.xticks([])
# plt.yticks([])
# plt.savefig('FTwp_dda_ddd.png'.format(i=i), dpi=400, transpent=True, bbox_inches='tight', pad_inches=0.02)

plt.show()

In [ ]:
# Save FT of the reconstructed wavelet packet data
plt.figure(figsize=(4.5, 3), dpi=100, layout='constrained')
# plt.plot(fq, abs(rfft(rwpn1)), '-b', label='DDA')
plt.plot(fq, abs(rfft(rwpn1)), '-r', label='DDD')
# plt.legend(frameon=False, fontsize=10)
plt.xlim([fq[0], fq[len(fq)-1]])
plt.ylim([-0.325, max(abs(rfft(rwpn1)))])
plt.xlabel('Frequency($Gauss^{-1}$)', fontsize=14, fontweight='bold')
plt.ylabel('Magnitude', fontsize=14, fontweight='bold')
# plt.xticks([])
# plt.yticks([])
# plt.savefig('FTwpn_ddd1.png', dpi=400, transparent=True, bbox_inches='tight', pad_inches=0.02)

plt.show()